In [1]:
%cd ~/meliad2
%env BUCKET meliad_eu2
%env LOGURU_LEVEL INFO

import seqio
from transformer import tasks
# task = seqio.get_mixture_or_task("pg19_tokens")


/home/ohadr/meliad2
env: BUCKET=meliad_eu2
env: LOGURU_LEVEL=INFO


2024-01-06 03:09:32.529212: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-01-06 03:09:32.529274: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-06 03:09:39.775781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-01-06 03:09:39.775851: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-06 03:09:39.775891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (v3-8-node-1): /proc/driver/nvidia/version does not exi

In [ ]:
/home/ohadr/.local/share/jupyter/runtime/kernel-v2-3064237Gr5Q5uZ3ZfPG.json

In [2]:

import functools
def delayed(function):
    """Decorator used to capture the arguments of a function."""

    def delayed_function(*args, **kwargs):
        return function, args, kwargs
    try:
        delayed_function = functools.wraps(function)(delayed_function)
    except AttributeError:
        " functools.wraps fails on some callable objects "
    return delayed_function
    
    

def apply_delayed(obj):
    function, args, kwargs = obj
    return function(*args, **kwargs)


In [3]:
import multiprocessing

def data_reader(delayed_source, output_queue, done_output_cnt):
    # Read data in chunks
    for data_chunk in apply_delayed(delayed_source):
        output_queue.put(data_chunk)
    done_output_cnt.value = 1


def map_process(input_queue, output_queue, map_function, done_input_cnt,done_output_cnt):
    while True:
        if done_input_cnt[0].value == done_input_cnt[1] and input_queue.empty():
            break
        data_chunk = input_queue.get()
        for processed_data in map_function(data_chunk):
            output_queue.put(processed_data)
    done_output_cnt[0].value += 1
                


def run_mapping_pipeline(data_source, map_functions, num_workers=10):
    num_stages = len(map_functions)

    # Queues for each stage, including the reader
    queues = [multiprocessing.Queue() for _ in range(num_stages+1)]
    
    done_cnt = [(multiprocessing.Value('i', 0),1)]
    for _ in range(num_stages):
        done_cnt.append((multiprocessing.Value('i', 0),num_workers))
    # Start the data reader worker
    reader_process = multiprocessing.Process(target=data_reader, args=(data_source, queues[0], done_cnt[0]))

    reader_process.start()

    # Create and start workers for each map stage
    workers = []
    for stage in range(num_stages):
        for _ in range(num_workers):
            worker = multiprocessing.Process(target=map_process, args=(queues[stage], queues[stage + 1], map_functions[stage], done_cnt[stage],done_cnt[stage+1]))
            worker.start()
            workers.append(worker)

    # Collect results and handle end signals
    end_signals = 0
    while end_signals < num_workers:  # Wait for all map workers to send end signal
        result = queues[-1].get()
        if result is None:
            end_signals += 1
        else:
            yield result

    # Clean up
    for queue in queues:
        queue.close()
        queue.join_thread()
    reader_process.join()
    for worker in workers:
        worker.join()

In [4]:
def extract_dpr_examples(element):
    neig = element["neig"]
    targets = element["targets"]
    chunk_id_list ,candidate_idx_list, candidate_rank_list = neig.reshape([-1,3]).T
    chunks = targets.reshape([-1,64])
    examples_dict = dict()
    
    for chunk_id,candidate_idx,candidate_rank in zip(chunk_id_list ,candidate_idx_list, candidate_rank_list):
        if chunk_id not in examples_dict:
            examples_dict[chunk_id] = {"question":chunks[chunk_id], "positive_ctxs":[], "hard_negative_ctxs":[]}
        if candidate_rank==0:
            examples_dict[chunk_id]["positive_ctxs"].append({"text":chunks[candidate_idx]})
        if candidate_rank==19:
            examples_dict[chunk_id]["hard_negative_ctxs"].append({"text":chunks[candidate_idx]})
    final_list = []
    for value in examples_dict.values():
        if len(value["positive_ctxs"])==1 and len(value["hard_negative_ctxs"])==1:
            final_list.append(value)
    return final_list

In [5]:
import jax
from functools import partial
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
def load_from_seqio(name, split):
    shard_id = jax.process_index()
    num_shards=jax.process_count()
    
    task = seqio.get_mixture_or_task(f"{name}neox_retro_nn20_f20_entirebook_qa_seq1024_16384_wtokens")
    dataset = task.get_dataset(split=split,
                                sequence_length=None,
                                shard_info=seqio.ShardInfo(shard_id,num_shards))
    yield from dataset.as_numpy_iterator()
delayed_dataset =  delayed(partial(load_from_seqio, name="codeparrot",split="train"))()

In [7]:

from tqdm import tqdm
for x in tqdm(run_mapping_pipeline(delayed_dataset, map_functions = [extract_dpr_examples],num_workers=50)):
    pass
    # print(x)


0it [00:00, ?it/s]2024-01-06 03:10:09.439871: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1 of 1000
2024-01-06 03:10:10.995797: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
676920it [02:18, 4892.26it/s] 


KeyboardInterrupt: 

In [ ]:
# from transformers import AutoTokenizer
# import datasets
# import numpy as np
# import seqio
# import jax
# from transformer import tasks
# from tqdm import tqdm
# from multiprocessing import Pool
# from functools import partial
# def extract_dpr_examples(element):
#     neig = element["neig"]
#     targets = element["targets"]
#     chunk_id_list ,candidate_idx_list, candidate_rank_list = neig.reshape([-1,3]).T
#     chunks = targets.reshape([-1,64])
#     # chunks = tokenizer.batch_decode(chunks, skip_special_tokens=True)
#     examples_dict = dict()
    
#     for chunk_id,candidate_idx,candidate_rank in zip(chunk_id_list ,candidate_idx_list, candidate_rank_list):
#         if chunk_id not in examples_dict:
#             examples_dict[chunk_id] = {"question":chunks[chunk_id], "positive_ctxs":[], "hard_negative_ctxs":[]}
#         if candidate_rank==0:
#             examples_dict[chunk_id]["positive_ctxs"].append({"text":chunks[candidate_idx]})
#         if candidate_rank==19:
#             examples_dict[chunk_id]["hard_negative_ctxs"].append({"text":chunks[candidate_idx]})
#     final_list = []
#     for value in examples_dict.values():
#         if len(value["positive_ctxs"])==1 and len(value["hard_negative_ctxs"])==1:
#             final_list.append(value)
#             # del examples_dict[chunk_id]
#     return final_list


In [2]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64/

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/


env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/06 00:10:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/06 00:10:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64/

# tfrecord_file = "gs://meliad_eu2/datasets/codeparrot-train-v2-near-dedup-safe/huggingface.co/datasets/iohadrubin/codeparrot-train-v2-near-dedup-safe/resolve/main/data/codeparrot/*.parquet"


from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, collect_list
num_partitions = 192

spark = SparkSession.builder \
    .appName("Read TFRecord with PySpark") \
    .config('spark.jars', 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar') \
    .config("spark.executor.memory", "256g") \
    .config("spark.driver.memory", "256g") \
    .config("spark.executor.memoryOverhead", "32g") \
    .config("spark.driver.memoryOverhead", "32g") \
    .config("spark.shuffle.service.enabled", "true") \
    .config("spark.shuffle.memoryFraction", "0.6") \
    .getOrCreate()

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")

from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, explode,udf
from pyspark.sql.types import ArrayType, BinaryType
import pandas as pd
import pickle



# Sample DataFrame to work with


# User-defined functions
def func1(i):
    return [i+1]

def func2(i):
    return [i*2]

from functools import wraps
def decorate_with_pickle(function):
    @wraps(function)
    def new_func(obj):
        _obj = pickle.loads(obj)
        res = [pickle.dumps(proc_obj) for proc_obj in  function(_obj)]
        return res 
    return udf(f=new_func,returnType=ArrayType(BinaryType()))

# Pandas UDF for transforming data
# def create_transform_udf(functions):
#     @pandas_udf(returnType=ArrayType(BinaryType()))
#     def transform_udf(series: pd.Series) -> pd.Series:
#         # Deserialize the received data
#         series = series.apply(lambda item: pickle.loads(item))
#         # Apply the transformations
#         for fn in functions:
#             series = series.apply(fn)
#         # Serialize the output and return
#         return series.apply(lambda item: pickle.dumps(item))
#     return transform_udf

def map_with_spark(data,functions):
    # Serialize the data
    functions = [decorate_with_pickle(fn) for fn in functions]
    # data = map(lambda x:[x], data)
    data = map(pickle.dumps, data)
    spark = SparkSession.builder.appName("pipeline").getOrCreate()
    df_serialized = spark.createDataFrame(data,  schema=BinaryType())

    # Apply UDF and explode
    # df_deserialized = df_serialized.withColumn("Output", explode(transform_udf(df_serialized["SerializedNumbers"])))
    for fn in functions:
        df_serialized = df_serialized.withColumn("value", explode(fn(df_serialized["value"])))

    # Iterator over the results
    results = (pickle.loads(row["value"]) for row in df_serialized.collect())
    # results = (row["value"] for row in df_serialized.collect())
    return results

# Test the function 
# data = [("object1", [1, 2, 3]), ("object2", [4, 5, 6])]
# data = [1,2,3,4,5,6]

def gen():
    for i in range(100000000000):
        yield i
data = iter(gen())
import tqdm
for cnt,output in enumerate(tqdm.tqdm(map_with_spark(data, functions = [func1, func2]))):
    pass
    # print(output)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data Processing Pipeline") \
    .getOrCreate()


In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, StringType

# Example map functions
def map_function_stage1(value):
    return value * 2

def map_function_stage2(value):
    return "Processed: " + str(value)

# Register UDFs (User Defined Functions)
map_udf_stage1 = udf(map_function_stage1, IntegerType())
map_udf_stage2 = udf(map_function_stage2, StringType())

# Assume 'column_name' is the name of the column you're processing
df = df.withColumn("stage1_output", map_udf_stage1(col("column_name")))
df = df.withColumn("stage2_output", map_udf_stage2(col("stage1_output")))
# Collect and print results (be cautious with large datasets)
results = df.select("stage2_output").collect()
for row in results:
    print(row['stage2_output'])


In [20]:
dataset = datasets.IterableDataset.from_generator(gen)

In [21]:
for i,x in enumerate(dataset.iter(10)):
    if i==10:
        break
    print(x)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-01-05 06:51:37.462049: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 2 of 1000
2024-01-05 06:51:38.756240: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


{'question': ['ing %s to %s" %\n                        (", ".join(files_to_copy), node_name))\n        for file_name in files_to_copy:\n          ctx.ssh.CopyFileToNode(node_name, file_name)\n\n  RunWhileClusterStopped', '(ToStdout, _RenewCryptoInner)\n\n  ToStdout("All requested certificates and keys have been replaced."\n           " Running \\"gnt-cluster verify\\" now is recommended.")\n\n  return 0\n\n\ndef RenewCrypto(opts, args):\n  """Renews cluster', ' certificates, keys and secrets.\n\n  """\n  return _RenewCrypto(opts.new_cluster_cert,\n                      opts.new_rapi_cert,\n                      opts.rapi_cert,\n                      opts.new_spice_cert,\n                      opts.spice_', 'cert,\n                      opts.spice_cacert,\n                      opts.new_confd_hmac_key,\n                      opts.new_cluster_domain_secret,\n                      opts.cluster_domain_secret,\n                      opts.force)\n\n\ndef SetClusterParams(opts,', ' args):\n 

In [ ]:
# output_types=datasets.Features({"question":datasets.Value("string"), "positive_ctxs":datasets.features.Sequence({"text":datasets.Value("string")}), "hard_negative_ctxs":datasets.features.Sequence({"text":datasets.Value("string")})})

In [60]:
dataset = list(dataset)

In [61]:
dataset[0]

{'question': ' = subnets\n            network_find.return_value = networks\n            yield ip_policy_update\n\n    def test_update_ip_policy_not_found(self):\n        with self._stubs(None):\n            with self.assertRaises(q_exc.IPPolicy',
 'positive_ctxs': ['s.ip_policy_update" % db_mod),\n        ) as (ip_policy_find, subnet_find, network_find, ip_policy_update):\n            ip_policy_find.return_value = ip_policy\n            subnet_find.return_value',
  '.1/32"])\n            self.assertEqual(resp["name"], "foo")\n            self.assertEqual(resp["tenant_id"], 1)\n\n\nclass TestQuarkUpdateIpPolicies(test_quark_plugin.TestQuarkPlugin):\n    @contextlib.contextmanager',
  'lib.nested(\n            mock.patch("%s.ip_policy_find" % db_mod),\n            mock.patch("%s.subnet_find" % db_mod),\n            mock.patch("%s.network_find" % db_mod),\n            mock.patch("%'],
 'hard_negative_ctxs': []}

In [ ]:
dataset.map(lambda x: x["question"])

In [27]:
neig.shape[0]/3

4736.0

In [29]:
chunk_size = 64
n_scored_nei = 20


In [30]:
nei_scores.shape[0]//(2*chunk_size*n_scored_nei)

2

In [21]:
chunks_in_document = targets.shape[0]//chunk_size

2

256.0

In [17]:
655360//(128*20)

256